# World countries data maniuplation 

In [ ]:
import sqlite3
import pandas as pd
import polars as pl
import requests
from io import StringIO

def get_data_from_url():
    # URL of the dataset
    url = "https://www.worldometers.info/world-population/population-by-country/"
   
    header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
    }
    
    response = requests.get(url, headers=header)
    
    
    # Read the HTML table from the URL using pandas
    tables = pd.read_html(StringIO(response.text))
    print(f'Length of tables: {len(tables)}')
    
    # Assuming the first table is what we want
    df_html = tables[0]
    return df_html

def convert_to_polars(pandas_df):
    # Convert to Polars DataFrame
    df_polars = pl.from_pandas(pandas_df)
    
    return df_polars

def write_to_excel(df_pandas, filename):
    # Write the DataFrame to an Excel file
    df_pandas.to_excel(filename, index=False)

    print(f'DataFrame written to {filename}')


    
get_data_from_url()





Length of tables: 1


,#,Country (or dependency),Population (2025),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Median Age,Urban Pop %,World Share
0,1,India,1463865525,0.89%,12929734,492,2973190,"−495,753",1.94,28.8,37.1%,17.78%
1,2,China,1416096094,−0.23%,"−3,225,184",151,9388211,"−268,126",1.02,40.1,67.5%,17.20%
2,3,United States,347275807,0.54%,1849236,38,9147420,1230663,1.62,38.5,82.8%,4.22%
3,4,Indonesia,285721236,0.79%,2233305,158,1811570,"−39,509",2.10,30.4,59.6%,3.47%
4,5,Pakistan,255219554,1.57%,3950390,331,770880,"−1,235,336",3.50,20.6,34.4%,3.10%
...,...,...,...,...,...,...,...,...,...,...,...,...
228,229,Montserrat,4359,−0.68%,−30,44,100,−9,1.45,41.6,11.5%,0.000053%
229,230,Falkland Islands,3469,−0.03%,−1,0,12170,−17,1.70,42.8,68%,0.000042%
230,231,Tokelau,2608,4.07%,102,261,10,53,2.57,27.3,NaN,0.000032%
231,232,Niue,1821,0.11%,2,7,260,3,2.46,35.7,44.5%,0.000022%


In [4]:
# write web data to excel
df_html = get_data_from_url()

filename = 'world_population.xlsx'
write_to_excel(df_html, filename)


#  Write data from internet do local db for safe keeping

conn = sqlite3.connect('countries.db')

df_html.to_sql('world_countries', conn, if_exists='replace', index=False)
print("Data written to SQLite database.")



Length of tables: 1
DataFrame written to world_population.xlsx
Data written to SQLite database.


In [10]:
# read data from excel and convert to polars dataframe
countries_df = pd.read_excel(filename)
countries_df.head(5)

def convert_to_polars(pandas_df):
    # Convert to Polars DataFrame
    df_polars = pl.from_pandas(pandas_df)
    
    return df_polars

countries_df_pl= convert_to_polars(countries_df)





In [14]:
# Filtering data 

print(countries_df_pl.columns)
df_filter_population = countries_df_pl.filter(pl.col('Population (2025)') > 5000000)

print("\n=== Filtered Data Countries with population > 10 mio ===")
df_filter_population.head()


['#', 'Country (or dependency)', 'Population (2025)', 'Yearly Change', 'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 'Migrants (net)', 'Fert. Rate', 'Median Age', 'Urban Pop %', 'World Share']

=== Filtered Data Countries with population > 10 mio ===


#,Country (or dependency),Population (2025),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Median Age,Urban Pop %,World Share
i64,str,i64,str,str,i64,i64,str,f64,f64,str,str
1,"""India""",1463865525,"""0.89%""","""12929734""",492,2973190,"""−495,753""",1.94,28.8,"""37.1%""","""17.78%"""
2,"""China""",1416096094,"""−0.23%""","""−3,225,184""",151,9388211,"""−268,126""",1.02,40.1,"""67.5%""","""17.20%"""
3,"""United States""",347275807,"""0.54%""","""1849236""",38,9147420,"""1230663""",1.62,38.5,"""82.8%""","""4.22%"""
4,"""Indonesia""",285721236,"""0.79%""","""2233305""",158,1811570,"""−39,509""",2.1,30.4,"""59.6%""","""3.47%"""
5,"""Pakistan""",255219554,"""1.57%""","""3950390""",331,770880,"""−1,235,336""",3.5,20.6,"""34.4%""","""3.10%"""


In [12]:
# Simpler column names

df = countries_df_pl.rename(lambda column_name: column_name.strip()) 
df = df.rename({'Country (or dependency)': "Country", 'Population (2025)': "Population", 'Yearly Change': 'Yearly_Change', 'Net Change': "Net_change", 'Density (P/Km²)':'Density', 'Land Area (Km²)': 'Land_Area', 'Migrants (net)': 'Migrants', 'Fert. Rate':'Fret_Rate', 'Median Age': 'Med_Age', 'Urban Pop %': 'Urban_Pop', 'World Share': "World_Share"})

print(df.columns)
df.head(5)

['#', 'Country', 'Population', 'Yearly_Change', 'Net_change', 'Density', 'Land_Area', 'Migrants', 'Fret_Rate', 'Med_Age', 'Urban_Pop', 'World_Share']


#,Country,Population,Yearly_Change,Net_change,Density,Land_Area,Migrants,Fret_Rate,Med_Age,Urban_Pop,World_Share
i64,str,i64,str,str,i64,i64,str,f64,f64,str,str
1,"""India""",1463865525,"""0.89%""","""12929734""",492,2973190,"""−495,753""",1.94,28.8,"""37.1%""","""17.78%"""
2,"""China""",1416096094,"""−0.23%""","""−3,225,184""",151,9388211,"""−268,126""",1.02,40.1,"""67.5%""","""17.20%"""
3,"""United States""",347275807,"""0.54%""","""1849236""",38,9147420,"""1230663""",1.62,38.5,"""82.8%""","""4.22%"""
4,"""Indonesia""",285721236,"""0.79%""","""2233305""",158,1811570,"""−39,509""",2.1,30.4,"""59.6%""","""3.47%"""
5,"""Pakistan""",255219554,"""1.57%""","""3950390""",331,770880,"""−1,235,336""",3.5,20.6,"""34.4%""","""3.10%"""


In [ ]:
# casting example

# Cast 'employee_id' to string for demonstration
df_casted = df.with_columns(
    pl.col("employee_id").cast(pl.Utf8).alias("employee_id_str")
)
print("\n=== Casted employee_id to string ===")
print(df_casted)


In [16]:


# Casting  Urban Pop % to float
# Strip the '%' character and convert to float
df_countries = df.with_columns(pl.col("Urban_Pop").str.strip_chars_end("%").cast(pl.Float32).alias("Urban_Pop_stripped"))
df_countries = df_countries.with_columns((pl.col("Urban_Pop_stripped")/100).alias("Urban_Pop_float"))

df_countries.head()



#,Country,Population,Yearly_Change,Net_change,Density,Land_Area,Migrants,Fret_Rate,Med_Age,Urban_Pop,World_Share,Urban_Pop_stripped,Urban_Pop_float
i64,str,i64,str,str,i64,i64,str,f64,f64,str,str,f32,f32
1,"""India""",1463865525,"""0.89%""","""12929734""",492,2973190,"""−495,753""",1.94,28.8,"""37.1%""","""17.78%""",37.099998,0.371
2,"""China""",1416096094,"""−0.23%""","""−3,225,184""",151,9388211,"""−268,126""",1.02,40.1,"""67.5%""","""17.20%""",67.5,0.675
3,"""United States""",347275807,"""0.54%""","""1849236""",38,9147420,"""1230663""",1.62,38.5,"""82.8%""","""4.22%""",82.800003,0.828
4,"""Indonesia""",285721236,"""0.79%""","""2233305""",158,1811570,"""−39,509""",2.1,30.4,"""59.6%""","""3.47%""",59.599998,0.596
5,"""Pakistan""",255219554,"""1.57%""","""3950390""",331,770880,"""−1,235,336""",3.5,20.6,"""34.4%""","""3.10%""",34.400002,0.344


In [ ]:
df_countries_mod = df_countries.select(['Country', 'Population', 'Land_Area', 'Density', 'Urban_Pop_float', 'Med_Age'])
df_countries_mod.head(5)

min_df = df_countries_mod.select(pl.col('Country'),pl.col('Population')).filter(pl.col('Population') == (pl.col('Population').min()))
max_df = df_countries_mod.select(pl.col('Country'),pl.col('Population')).filter(pl.col('Population') == (pl.col('Population').max()))
median_df = df_countries_mod.select(pl.col('Country'),pl.col('Population')).filter(pl.col('Population') == (pl.col('Population').median()))


# Concatenate the DataFrames
df_stats = pl.concat([min_df, max_df, median_df], how='vertical')
df_stats_pd = df_stats.to_pandas()




print("\n=== Min, Max, Median Population ===") 
print(df_stats)



=== Min, Max, Median Population ===
shape: (3, 2)
┌──────────┬────────────┐
│ Country  ┆ Population │
│ ---      ┆ ---        │
│ str      ┆ i64        │
╞══════════╪════════════╡
│ Holy See ┆ 501        │
│ India    ┆ 1463865525 │
│ Liberia  ┆ 5731206    │
└──────────┴────────────┘


In [17]:
# Sort countries by population Ascending

df_pop_sorted = df.sort('Population', descending=False)

df_pop_sorted = df_pop_sorted.select(['Country', 'Population'])
print("\n=== Sorted Data by Population Ascending ===")

print(df_pop_sorted)


def write_to_json(df, filename):
    # Write the DataFrame to a JSON file
    df.write_json(filename)
    print(f'DataFrame written to {filename}')


=== Sorted Data by Population Ascending ===
shape: (233, 2)
┌──────────────────┬────────────┐
│ Country          ┆ Population │
│ ---              ┆ ---        │
│ str              ┆ i64        │
╞══════════════════╪════════════╡
│ Holy See         ┆ 501        │
│ Niue             ┆ 1821       │
│ Tokelau          ┆ 2608       │
│ Falkland Islands ┆ 3469       │
│ Montserrat       ┆ 4359       │
│ …                ┆ …          │
│ Pakistan         ┆ 255219554  │
│ Indonesia        ┆ 285721236  │
│ United States    ┆ 347275807  │
│ China            ┆ 1416096094 │
│ India            ┆ 1463865525 │
└──────────────────┴────────────┘


In [36]:
# write to json


def write_to_json(df, filename):
    # Write the DataFrame to a JSON file
    df.write_json(filename)
    print(f'DataFrame written to {filename}')

filename = 'world_population.json'
write_to_json(df_pop_sorted, filename)

DataFrame written to world_population.json
